In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# ------------------------
# 1. Load dataset
# ------------------------
df = pd.read_csv("Health_Quality_Dataset.csv")

# ------------------------
# 2. Features & Target
# ------------------------
X = df.drop(columns=["true_prob"])
y = df["true_prob"]

# Identify categorical & numeric columns
categorical_cols = ["sex", "sanitation", "water_source"]
numeric_cols = [c for c in X.columns if c not in categorical_cols]

# ------------------------
# 3. Preprocessing: One-hot encode categorical vars
# ------------------------
preprocessor = ColumnTransformer(
    transformers=[
        ("cat", OneHotEncoder(handle_unknown="ignore"), categorical_cols),
        ("num", "passthrough", numeric_cols)
    ]
)

# ------------------------
# 4. Train/Test Split
# ------------------------
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [2]:
# ------------------------
# 5. Build Random Forest Pipeline
# ------------------------
rf_model = Pipeline(steps=[
    ("preprocessor", preprocessor),
    ("regressor", RandomForestRegressor(
        n_estimators=200,
        max_depth=12,
        random_state=42,
        n_jobs=-1
    ))
])
# Fit model
rf_model.fit(X_train, y_train)

# ------------------------
# 6. Evaluation
# ------------------------
y_pred = rf_model.predict(X_test)

print("R² Score:", r2_score(y_test, y_pred))
print("MAE:", mean_absolute_error(y_test, y_pred))
# Calculate RMSE by taking the square root of the MSE
print("RMSE:", np.sqrt(mean_squared_error(y_test, y_pred)))

# ------------------------
# 7. Feature Importance (approx)
# ------------------------
# To get feature importances, extract from the trained regressor
regressor = rf_model.named_steps["regressor"]

# Get one-hot encoded feature names
ohe = rf_model.named_steps["preprocessor"].named_transformers_["cat"]
ohe_features = ohe.get_feature_names_out(categorical_cols)
all_features = np.concatenate([ohe_features, numeric_cols])

importances = regressor.feature_importances_
feat_imp = pd.DataFrame({"feature": all_features, "importance": importances})
feat_imp = feat_imp.sort_values("importance", ascending=False)

print("\nTop 10 Important Features:")
print(feat_imp.head(10))


R² Score: 0.8628063947863092
MAE: 0.031479130729042
RMSE: 0.04044361076109148

Top 10 Important Features:
            feature  importance
17         vomiting    0.234174
16         diarrhea    0.227126
20         jaundice    0.200509
18            fever    0.156700
15              age    0.028468
2   sanitation_good    0.021017
19          fatigue    0.017904
3   sanitation_poor    0.017791
22    loss_appetite    0.013784
23     muscle_aches    0.011096


In [3]:
from xgboost import XGBRegressor

xgb_model = Pipeline(steps=[
    ("preprocessor", preprocessor),
    ("regressor", XGBRegressor(
        n_estimators=300,
        learning_rate=0.05,
        max_depth=6,
        subsample=0.8,
        colsample_bytree=0.8,
        random_state=42
    ))
])

xgb_model.fit(X_train, y_train)
y_pred_xgb = xgb_model.predict(X_test)

print("XGBoost R²:", r2_score(y_test, y_pred_xgb))
print("XGBoost MAE:", mean_absolute_error(y_test, y_pred_xgb))
print("XGBoost RMSE:", np.sqrt(mean_squared_error(y_test, y_pred_xgb)))


XGBoost R²: 0.8791551226249125
XGBoost MAE: 0.029363521593616018
XGBoost RMSE: 0.03795745713675475


In [4]:
from lightgbm import LGBMRegressor

lgb_model = Pipeline(steps=[
    ("preprocessor", preprocessor),
    ("regressor", LGBMRegressor(
        n_estimators=300,
        learning_rate=0.05,
        max_depth=-1,
        num_leaves=31,
        subsample=0.8,
        colsample_bytree=0.8,
        random_state=42
    ))
])

lgb_model.fit(X_train, y_train)
y_pred_lgb = lgb_model.predict(X_test)

print("LightGBM R²:", r2_score(y_test, y_pred_lgb))
print("LightGBM MAE:", mean_absolute_error(y_test, y_pred_lgb))
print("LightGBM RMSE:", np.sqrt(mean_squared_error(y_test, y_pred_lgb)))


[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000784 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 128
[LightGBM] [Info] Number of data points in the train set: 8000, number of used features: 24
[LightGBM] [Info] Start training from score 0.767528
LightGBM R²: 0.8820813995754307
LightGBM MAE: 0.02901926615722816
LightGBM RMSE: 0.037495067999284884


/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(


In [5]:
from sklearn.ensemble import ExtraTreesRegressor

et_model = Pipeline(steps=[
    ("preprocessor", preprocessor),
    ("regressor", ExtraTreesRegressor(
        n_estimators=300,
        max_depth=None,
        min_samples_split=2,
        random_state=42,
        n_jobs=-1
    ))
])

et_model.fit(X_train, y_train)
y_pred_et = et_model.predict(X_test)

print("ExtraTrees R²:", r2_score(y_test, y_pred_et))
print("ExtraTrees MAE:", mean_absolute_error(y_test, y_pred_et))
print("ExtraTrees RMSE:", np.sqrt(mean_squared_error(y_test, y_pred_et)))


ExtraTrees R²: 0.8007507451317124
ExtraTrees MAE: 0.037524568792806676
ExtraTrees RMSE: 0.04873953040910984


In [7]:
import joblib

# Save Random Forest
joblib.dump(rf_model, "rf_health_model.pkl")

# Save XGBoost
joblib.dump(xgb_model, "xgb_health_model.pkl")

# Save LightGBM
joblib.dump(lgb_model, "lgb_health_model.pkl")

['lgb_health_model.pkl']